In [ ]:
# Code for screenshot of highlighted airway tree

import SimpleITK as sitk
import nibabel as nib
from mayavi import mlab
import numpy as np

airway_file = r'z:\E-Kelly-VidaSegmentals&Sublobes\2- 2AdditionalScans\Kelly Done\SPIROMICS-51287502\A1-Recentered\FlippedAndRecenteredAirwClr.nii.gz'

# Loads image file
airway_img = nib.load(airway_file)

# Turns image data into numpy array
airway_image_data = airway_img.get_fdata()

# Identify which segmentations should be which color
red_index = [1, 2, 3, 8, 9, 12, 13, 33, 40, 48, 53, 55, 73, 185]

orange_index = [64, 65, 76, 77, 90, 91, 104, 105, 246, 247]

# Makes locations of previous segmentations 1 and all else 0
red_segments = np.isin(airway_image_data, red_index).astype(int)

orange_segments = np.isin(airway_image_data, orange_index).astype(int)

# Makes array that includes all possible segments
all_segments_index = np.arange(1, 308)

# Makes array of all segments not included in the orange or red indicies
background_index = np.setdiff1d(all_segments_index, np.union1d(red_index, orange_index))

# Makes locations of background segments 1 and all else 0
background_segments = np.isin(airway_image_data, background_index).astype(int)

# Find Segmentation Center
model = sitk.ReadImage(airway_file)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

mlab.figure(bgcolor=(1, 1, 1))

# This 600 is 60 cm away from the coronal plane from behind
x, y, z = center[0], center[1], center[2]
ypt = y + 600
camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])


# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

mlab.gcf().scene.camera.view_angle = 23 # Adjust the FOV angle
mlab.gcf().scene.camera.clipping_range = (1, 10000)  # Adjust near and far clipping planes

red_mask = mlab.contour3d(red_segments, contours=[0.5], opacity=1, color=(1, 0, 0))

orange_mask = mlab.contour3d(orange_segments, contours=[0.5], opacity=1, color=(1, 0.5, 0))

background_mask = mlab.contour3d(background_segments, contours=[0.5], color=(1, 0.75, 0), opacity=0.25)

screenshot_filename = r"Z:\Q- Figures\T32-Poster\Allyson\Experiment\Case 24\colored_airway_screenshot.png"

mlab.savefig(screenshot_filename, size=(800, 800))